# Embedded ML - Lab 2.3: TensorFlow Lite Micro

Tensor Flow Lite Micro (TFLM) is a library that aims to run ML models efficiently on embedded systems. It's a C++ library that provides a version of the TensorFlow Lite interpreter that supports less types of operations and uses less memory. The library also provides helper functions for data pre- and post-processing.

In [2]:
# To run this notebook, locally as a jupyter notebook, you need to install thhe proper packages.
# follow the instructions below to set up your environment.


# 1. Crea un entorno virtual usando conda o venv
#    Por ejemplo, usando conda:
#        conda create -n [myenv] python=3.8
#        conda activate [myenv]
#    O usando venv:
#        python3 -m venv [myenv]
#        source [myenv]/bin/activate
#
# 2. Activa el entorno virtual
# 3. Instala los paquetes requeridos usando pip
# 4. Ejecuta el notebook


# Instala los paquetes requeridos:
%pip install numpy -q
%pip install pandas -q
%pip install matplotlib -q
%pip install tensorflow -q
%pip install scikit-learn -q
%pip install tensorflow-hub -q
%pip install tensorflow-datasets -q
%pip install tensorflow-estimator -q
%pip install roboflow -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [4]:
from roboflow import Roboflow


rf = Roboflow(api_key="w5hJLcBvvd3Y3rBjxkkU")
project = rf.workspace("eml-pb5ag").project("rock-paper-scissors-vwkqp")
version = project.version(1)
dataset = version.download("folder")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Rock-Paper-Scissors-1 in folder:: 100%|██████████| 2939/2939 [00:00<00:00, 8201.19it/s]


### Learning outcomes


* Explain the basic concepts associated with TFLM
* Use the API to implement the TFLM workflow for an embedded application
* Execute TFLM code on a microcontroller-based embedded system

### TensorFlow Lite Micro workflow

TFLM's high-level workflow is rather simple:
* Generate a small TensorFlow model that can fit your target device and contains supported operations.
* Convert to a TensorFlow Lite model using the TensorFlow Lite converter, applying quantization if required.
* Convert to a C byte array using standard tools and stored it in the read-only program memory on device.
* Run inference on device using the TFLM C++ library and process the results.

### Hello World and Hello Human

After installing the Arduino IDE and the board files, you should install the Harvard_TinyMLx library that contains the TensorFlow Lite Micro and other resources and examples to build ML apps with Arduino and TFLM. Later on, depending on the application you want to build and the specific hardware to be used, you should install the propper peripheral drivers for communication, sensing and actuating.



*   Install Arduino IDE 2 from: https://downloads.arduino.cc/arduino-ide/arduino-ide_2.3.2_Linux_64bit.AppImage
*   From the boards manager install: Arduino Mbed OS Nano boards
*   Allow the linux user to access serial port: `sudo usermod -a -G dialout \<username\>` (reboot afterwards)
*   From the library manager install: Harvard_TinyMLx




Now open the **Hello World** example from the Harvard_TinyMLx library File->Examples->Harvard_TinyMLx in Arduino IDE (also available in [this repo](https://github.com/tinyMLx/arduino-library/tree/main/examples/hello_world)), compile it and run it on the microcontroller board. It is an ML model to predict a sine wave that is used to dim on and off an LED. The Arduino IDE serial monitor should also show interger numbers up and down trying to model a sine wave. This is a test app to make sure that the basic HW and SW elements, including TFLM, are working.

Inspect the code to make sure you identify and understand the main parts of the workflow.

Running on-device inference using the TFLM C++ library usually involves:

* Include the library headers
* Include the model header
* Load a model
* Instantiate operations resolver
* Allocate memory
* Instantiate interpreter
* Read and pre-process input data
* Provide inputs to the allocated tensors
* Run inference
* Get results from the output tensors
* Take action based on outputs

After you have succesfully run the Hello World example, move on to running the **Person Detection** example from the same library. Explore the code in detail to understand how to handle the **camera**.

### TinyML application development

ML applications that run on embeded systems with very limited resources are often called TinyML. In this lab the goal is to develop a simple TinyML application that uses computer vision up to its deployment on the target embedded device: **Arduino Nano 33 BLE.**

Follow these steps in order to develop your TinyML application:

1. Select two visual objects that are radically different and  assemble a dataset that contains at least hundreds or thousands of examples. You can create the images yourslef or extract them from a public database and apply data augmentation.

2. Design and train a model to classify between the two chosen objects. You can build a dense or CNN model from scratch, or use transfer learning, but you should always keep in mind the very limited memory resources of the target device as well as the image properties of the embedded camera.

3.   Export the trained model to a file and convert it to a C header by running the following linux command: `xxd -i converted_model.tflite > converted_model_data.h`

4.   Develop an Arduino code based on the Hello World and Person Detection examples, to detect whether any of the two objects are present on the camera view. Indicate the result through the RGB LED.

Include in your notebook submission both the code you developed to build the model as well as the C++ codes for the MCU.

In [6]:
# Import the necessary libraries

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


import os
import logging
import sys
import contextlib

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel(logging.ERROR)

@contextlib.contextmanager
def suppress_stdout():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:
            yield
        finally:
            sys.stdout = old_stdout

In [ ]:
# Tensor flow model 

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(640, 480, 1)),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(8, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3, activation='softmax'),  
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 638, 478, 8)    │            80 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 319, 239, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 317, 237, 8)    │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 158, 118, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 149152)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │       447,459 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 448,123 (1.71 MB)

 Trainable params: 448,123 (1.71 MB)

 Non-trainable params: 0 (0.00 B)